In [30]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [94]:
survey_year = '2018'
survey_type = 'Climate Change'
cities_df = pd.read_csv('Cities/Cities Responses/'+survey_year+'_Full_Cities_Dataset.csv')
cities_info_df = pd.read_csv('Cities/Cities Disclosing/'+survey_year+'_Cities_Disclosing_to_CDP.csv')
corps_cc_df = pd.read_csv('Corporations/Corporations Responses/'+survey_type+'/'+survey_year+'_Full_Climate_Change_Dataset.csv')
na_hq_df = pd.read_csv('Supplementary Data/Locations of Corporations/NA_HQ_public_data.csv')
corps_cc_info = pd.read_csv('Corporations/Corporations Disclosing/'+survey_type+'/'+survey_year+'_Corporates_Disclosing_to_CDP_Climate_Change.csv')
census_df = pd.read_csv('Supplementary Data/CDC 500 Cities Census Tract Data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.csv')
social_df = pd.read_csv('Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US.csv')
social_county_df = pd.read_csv('Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US_COUNTY.csv')
uscities_df_orig = pd.read_csv('Supplementary Data/Simple Maps US Cities Data/uscities.csv')

### Cleaning City Data

In [95]:
city_names_df = cities_info_df[['Account Number','City']]
city_names_df[city_names_df['City'] == 'Santiago']
city_names_df.loc[city_names_df['Account Number'] == 60393, 'City'] = 'Santiago Government'

/Users/Jenny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<span style="color:red">To do: Look at other year surveys</span>

In [100]:
cities_df_clean = pd.merge(cities_df,city_names_df,how='left',on='Account Number')
cities_df_clean['Organization'] = cities_df_clean['City']
cities_df_clean.drop(labels='City', axis="columns", inplace=True)
cities_df_clean = cities_df_clean.rename(columns={'Organization':'City'})
cities_df_clean.head()

Questionnaire  Year Reported to CDP  Account Number      City  \
0  CDP Cities 2018                  2018            3203   Chicago   
1  CDP Cities 2018                  2018           50555  Hamilton   
2  CDP Cities 2018                  2018           50392   Vitória   
3  CDP Cities 2018                  2018           36522  Verbania   
4  CDP Cities 2018                  2018           35873  Medellín   

                    Country     CDP Region                  Parent Section  \
0  United States of America  North America                           Water   
1                    Canada  North America                 Climate Hazards   
2                    Brazil  Latin America  Emissions Reduction: City-wide   
3                     Italy         Europe                        Strategy   
4                  Colombia  Latin America                 Climate Hazards   

                                   Section Question Number  \
0                               Wastewater            16.1   
1                          Climate Hazards            2.2a   
2  Emissions Reduction Actions : City-wide             8.4   
3                                   Energy             9.0   
4                          Climate Hazards            2.2a   

                                       Question Name  Column Number  \
0  Please provide the percentage breakdown of the...              1   
1  Please list the most significant climate hazar...              5   
2  What actions is your city taking to reduce emi...              1   
3  Please indicate the energy mix of electricity ...              2   
4  Please list the most significant climate hazar...              3   

                            Column Name  Row Number  \
0    Percentage of wastewater collected           5   
1                 Probability of hazard           4   
2  Emissions reduction project activity           3   
3                                   Gas           1   
4                   Magnitude of impact           6   

                        Row Name                      Response Answer  \
0       Other type of wastewater                                  NaN   
1                            NaN                          Medium High   
2                            NaN  On-site renewable energy generation   
3  Energy consumption percentage                                  NaN   
4                            NaN                              Serious   

  Comments File Name             Last update  
0      NaN       NaN  24/06/2020 05:28:18 AM  
1      NaN       NaN  24/06/2020 05:28:18 AM  
2      NaN       NaN  24/06/2020 05:28:18 AM  
3      NaN       NaN  24/06/2020 05:28:18 AM  
4      NaN       NaN  24/06/2020 05:28:18 AM

### Cleaning Corporations Data

In [34]:
# adding headquarter location data
hq_df = na_hq_df[(na_hq_df['survey_year'] == survey_year) & (na_hq_df['theme'] == survey_type)]
hq_df = hq_df[['account_number','hq_country','address_city','address_state','address_lat','adress_long']]
hq_df = hq_df.rename(columns={'address_city':'hq_city','address_state':'hq_state','address_lat':'hq_lat','adress_long':'hq_long'})
corps_cc_hq_df = pd.merge(corps_cc_df, hq_df, how='left', on='account_number')

# adding industry data
corps_cc_info = corps_cc_info[['account_number','activities','sectors','industries','primary_activity','primary_sector','primary_industry','primary_questionnaire_sector']]
corps_cc_full = pd.merge(corps_cc_hq_df, corps_cc_info, how='left', on='account_number')
corps_cc_full.head()

/Users/Jenny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


account_number            organization  survey_year response_received_date  \
0             912  Archer Daniels Midland         2018   2018-09-04T00:00:00Z   
1             912  Archer Daniels Midland         2018   2018-09-04T00:00:00Z   
2             912  Archer Daniels Midland         2018   2018-09-04T00:00:00Z   
3             912  Archer Daniels Midland         2018   2018-09-04T00:00:00Z   
4             912  Archer Daniels Midland         2018   2018-09-04T00:00:00Z   

  accounting_period_to  ors_response_id       submission_date page_name  \
0                  NaN            13370  2018-10-24T16:36:50Z      C0.2   
1                  NaN            13370  2018-10-24T16:36:50Z      C0.4   
2                  NaN            13370  2018-10-24T16:36:50Z      C0.2   
3                  NaN            13370  2018-10-24T16:36:50Z      C0.3   
4                  NaN            13370  2018-10-24T16:36:50Z      C0.5   

        module_name question_number  ... hq_state  hq_lat hq_long  \
0  C0. Introduction            C0.2  ...      NaN     NaN     NaN   
1  C0. Introduction            C0.4  ...      NaN     NaN     NaN   
2  C0. Introduction            C0.2  ...      NaN     NaN     NaN   
3  C0. Introduction            C0.3  ...      NaN     NaN     NaN   
4  C0. Introduction            C0.5  ...      NaN     NaN     NaN   

                                          activities  \
0   Agricultural products wholesale, Banks, Biofu...   
1   Agricultural products wholesale, Banks, Biofu...   
2   Agricultural products wholesale, Banks, Biofu...   
3   Agricultural products wholesale, Banks, Biofu...   
4   Agricultural products wholesale, Banks, Biofu...   

                                             sectors  \
0   Chemicals, Financial services, Food & beverag...   
1   Chemicals, Financial services, Food & beverag...   
2   Chemicals, Financial services, Food & beverag...   
3   Chemicals, Financial services, Food & beverag...   
4   Chemicals, Financial services, Food & beverag...   

                                          industries    primary_activity  \
0   Food, beverage & agriculture, Materials, Reta...  Oilseed processing   
1   Food, beverage & agriculture, Materials, Reta...  Oilseed processing   
2   Food, beverage & agriculture, Materials, Reta...  Oilseed processing   
3   Food, beverage & agriculture, Materials, Reta...  Oilseed processing   
4   Food, beverage & agriculture, Materials, Reta...  Oilseed processing   

               primary_sector              primary_industry  \
0  Food & beverage processing  Food, beverage & agriculture   
1  Food & beverage processing  Food, beverage & agriculture   
2  Food & beverage processing  Food, beverage & agriculture   
3  Food & beverage processing  Food, beverage & agriculture   
4  Food & beverage processing  Food, beverage & agriculture   

  primary_questionnaire_sector  
0     Food, beverage & tobacco  
1     Food, beverage & tobacco  
2     Food, beverage & tobacco  
3     Food, beverage & tobacco  
4     Food, beverage & tobacco  

[5 rows x 32 columns]

### Characterstics of Each City

In [107]:
intro_questions = ('0.1','0.3','0.5','0.6','0.7','0.8')

intro_df = cities_df_clean[(cities_df_clean['Question Number'].isin(intro_questions))][['City','Column Name','Response Answer']]
intro_df = intro_df.pivot(index='City', columns='Column Name', values='Response Answer')

cities_desc_df = cities_df_clean[(cities_df_clean['Question Number'] == '0.1') & (cities_df_clean['Column Number'] == 1)].sort_values('Country')[['City','Country','CDP Region']]

pd.merge(cities_desc_df,intro_df,how='left',on='City').sort_values('Country')


City                   Country  \
0    San Isidro (Argentina)                 Argentina   
1                   Ataliva                 Argentina   
2                   Mendoza                 Argentina   
3             Vicente López                 Argentina   
4                Rio Grande                 Argentina   
..                      ...                       ...   
390              Dublin, CA  United States of America   
484             Ho Chi Minh                  Viet Nam   
486                  Kasama                    Zambia   
485                Mazabuka                    Zambia   
487                  Harare                  Zimbabwe   

                     CDP Region  \
0                 Latin America   
1                 Latin America   
2                 Latin America   
3                 Latin America   
4                 Latin America   
..                          ...   
390               North America   
484  Southeast Asia and Oceania   
486                      Africa   
485                      Africa   
487                      Africa   

                               Administrative boundary  \
0                                  City / Municipality   
1                                        Other: Pueblo   
2                                  City / Municipality   
3                                  City / Municipality   
4                                  City / Municipality   
..                                                 ...   
390                                City / Municipality   
484                                       Special city   
486                                City / Municipality   
485                                City / Municipality   
487  Local government area within a city / metropol...   

    Annual operating budget Average altitude (m)  \
0                5998952920                   20   
1                     24000                   89   
2                   2200000                  NaN   
3               4354985.553                   11   
4               63335000000                   18   
..                      ...                  ...   
390               109400000                  111   
484                     NaN                    6   
486                34800000                  NaN   
485                24996530                 1200   
487                     345                 1500   

    Average annual temperature (in Celsius) Budget year end Budget year start  \
0                                        18      2018-12-31        2018-01-01   
1                                      18.6      2018-12-31        2018-01-01   
2                                        24      2018-12-31        2018-01-01   
3                                      16.6      2017-12-31        2017-01-01   
4                                         6      2017-12-31        2017-01-01   
..                                      ...             ...               ...   
390                                    14.7      2019-06-30        2018-07-01   
484                                      26             NaN               NaN   
486                                      24      2018-12-31        2018-02-01   
485                                    22.3      2019-12-31        2017-01-01   
487                                      18      2017-12-31        2017-01-01   

    Current population  ... Land area (in square km) Latitude (e.g. 41.25)  \
0              292.878  ...                       50                 34.28   
1                 2500  ...                      155                    31   
2               115041  ...                   106.07             -32.89084   
3               270929  ...                    34.45             58.483333   
4                84246  ...                    12181             -27.18333   
..                 ...  ...                      ...                   ...   
390              63241  ...                    15.23                  37.7  

In [37]:
#count number of cities by country
cities_desc_df[['Organization','Country']].groupby('Country').count().sort_values('Organization',ascending=False).head(20)

#see which cities are in the census data
cities_desc_df[cities_desc_df['Organization'].isin(census_df.PlaceName.unique().tolist())][['Organization','Country']].groupby('Country').count()

#see which cities are in the social vulnerability data -> see lower cell b/c have to match counties with cities first


Organization
Country                                                         
Canada                                                         2
Greece                                                         1
Italy                                                          1
United Kingdom of Great Britain and Northern Ir...             2
United States of America                                      75

### Cleaning Social Vulnerability Data

In [92]:
social_county_df["STATE"] = social_county_df["STATE"].str.lower()
social_county_df["COUNTY"] = social_county_df["COUNTY"].str.lower()

uscities_df = uscities_df_orig.copy()
uscities_df["state_name"] = uscities_df["state_name"].str.lower()
uscities_df["county_name"] = uscities_df["county_name"].str.lower()
uscities_df = uscities_df.rename(columns = {'county_name':'COUNTY','state_name':'STATE','city':'City'})
social_cities_df = pd.merge(social_county_df,uscities_df[['City','STATE','COUNTY']],on=['STATE','COUNTY'],how='right')

City State
0       San Jose    CA
1      Rochester    NY
4           Cork   NaN
5     Evansville    IN
6        Atlanta    GA
...          ...   ...
3280    Danville    CA
3292   Milwaukee    WI
3295     Calgary    AB
3297     Waltham    MA
3298   Oak Brook    IL

[2344 rows x 2 columns]

In [41]:
#see which cities are in the social vulnerability data -> note they should all be in the US but some cities have same name in other countries

cities_desc_df[cities_desc_df['Organization'].isin(social_cities_df.City.unique().tolist())][['Organization','Country']].groupby('Country').count()

Organization
Country                                                         
Argentina                                                      1
Australia                                                      2
Brazil                                                         2
Canada                                                         9
Chile                                                          1
Colombia                                                       1
Comoros                                                        1
Costa Rica                                                     1
Denmark                                                        1
France                                                         2
Germany                                                        2
Gibraltar                                                      1
Greece                                                         1
India                                                          1
Italy                                                          3
Netherlands                                                    1
New Zealand                                                    1
Norway                                                         1
Peru                                                           1
Poland                                                         1
Portugal                                                       1
South Africa                                                   1
Spain                                                          1
Sweden                                                         1
United Kingdom of Great Britain and Northern Ir...             6
United States of America                                     113

### Creating Cities and States Frame

In [108]:
# state abbreviation dictionary
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# map dict to clean full state names to abbreviations
na_hq_df['state'] = na_hq_df['address_state'].map(us_state_abbrev)

# infill non-matched from dict
na_hq_df['state'] = na_hq_df['state'].fillna(na_hq_df['address_state'])

na_hq_df['state'] = na_hq_df['state'].replace({'ALBERTA':'AB'})
na_hq_df['address_city'] = na_hq_df['address_city'].replace({'CALGARY':'Calgary'})

cities_states_df = na_hq_df[['address_city','state']]
cities_states_df = cities_states_df[cities_states_df['address_city'].notnull()]

cities_states_df = cities_states_df.rename(columns={'address_city':'City', 'state':'State'})
cities_states_df

City State
0       San Jose    CA
1      Rochester    NY
4           Cork   NaN
5     Evansville    IN
6        Atlanta    GA
...          ...   ...
3280    Danville    CA
3292   Milwaukee    WI
3295     Calgary    AB
3297     Waltham    MA
3298   Oak Brook    IL

[2344 rows x 2 columns]

## Finding Cities Most At Risk

#### Introduction

The relationship between climate change and the most disadvantaged groups in society is well known to be characterized as a **positive feedback loop**. Initial inequality allows the most disadvantaged groups to be impacted disproportionately worse by the impacts of climate change, which results in increased subsequent inequality. 

Finding the cities that are most at risk of this positive feedback loop can be of use to both local and national governments, as it would provide information for what parts of the state and country to direct funding and resources to prevent the cycle from perpetuating. 

In addition, as we have seen with the current pandemic, governments have difficulty providing resources to the corporations that need it most and instead opt for widespread injection of stimulus to corporate balance sheets (**might need link here**). Knowing which companies have locations in the areas that are most disadvantaged (from both a socioeconomic and environmental standpoint), would provide a more direct tool to direct resources.

To find cities that are most at risk from both ends of climate change and inequality, we can define metrics that highlight whether a city is facing social and economic disadvantages in comparison to other cities, across the different types of climate hazards that are likely to prevalent in the future. 


### Creating Social Vulnerability Metrics

In [70]:
def normalize(df,var):
    min_var = df[var].min()
    max_var = df[var].max()
    diff = max_var - min_var
    df['new_var'] = (df[var] - min_var) / diff
    return df

#poverty metric
sc_metrics = social_cities_df.copy()
sc_pov = sc_metrics[['STATE','COUNTY','City','EP_POV']][sc_metrics['EP_POV'] >= 0]
sc_pov = normalize(sc_pov,'EP_POV').rename(columns={'new_var':'EP_POV_std'})
sc_pov.sort_values('EP_POV_std')

#unemployment metric

#disadvantaged score




Year Reported to CDP  Account Number  \
0                    2018           58671   
1                    2018           59167   
2                    2018           50553   
3                    2018          826407   
4                    2018           60375   
..                    ...             ...   
636                  2018           74631   
637                  2018           60898   
638                  2018           60638   
639                  2018           35995   
640                  2018          826114   

                                    Organization                        City  \
0      Helsingør Kommune / Elsinore Municipality                   Helsingør   
1                   Municipalidad de Providencia                 Providencia   
2                                 City of Fresno                      Fresno   
3                         Munícipio de Mirandela                   Mirandela   
4    Gobiernación del Archipiélago de San Andrés  Archipiélago de San Andrés   
..                                           ...                         ...   
636                                   Lubumbashi                  Lubumbashi   
637          Ayuntamiento de Naucalpan de Juárez         Naucalpan de Juárez   
638                      Walvis Bay Municipality                  Walvis Bay   
639                             City of Brasília                    Brasília   
640                    Ayuntamiento de Degollado                   Degollado   

                              Country     CDP Region Reporting Authority  \
0                             Denmark         Europe          CDP Cities   
1                               Chile  Latin America          CDP Cities   
2            United States of America  North America          CDP Cities   
3                            Portugal         Europe          CDP Cities   
4                            Colombia  Latin America          CDP Cities   
..                                ...            ...                 ...   
636  Democratic Republic of the Congo         Africa          CDP Cities   
637                            Mexico  Latin America          CDP Cities   
638                           Namibia         Africa          CDP Cities   
639                            Brazil  Latin America          CDP Cities   
640                            Mexico  Latin America          CDP Cities   

         Access First Time Discloser  Population  Population Year  \
0        public                   No     62686.0             2017   
1        public                   No    126487.0             2012   
2        public                  Yes    527438.0             2017   
3        public                  Yes     11852.0             2011   
4        public                   No    120000.0             2017   
..          ...                  ...         ...              ...   
636      public                  Yes   5652000.0             2016   
637      public                   No         NaN                0   
638  non public                   No    100000.0             2016   
639  non public                   No      3013.0             2017   
640      public                  Yes         NaN                0   

                 City Location              Last update  
0      POINT (12.5921 56.0308)  2020-06-25T04:52:49.050  
1    POINT (-70.6092 -33.4313)  2020-06-25T04:52:49.050  
2     POINT (-119.773 36.7468)  2020-06-25T04:52:49.050  
3        POINT (-7.1753 41.51)  2020-06-25T04:52:49.050  
4     POINT (-81.6976 12.5794)  2020-06-25T04:52:49.050  
..                         ...                      ...  
636             POINT (-27 11)  2020-06-25T04:52:49.050  
637   POINT (-99.2453 19.4631)  2020-06-25T04:52:49.050  
638   POINT (12.5247 -22.9389)  2020-06-25T04:52:49.050  
639  POINT (-47.8822 -15.7942)  2020-06-25T04:52:49.050  
640   POINT (-102.132 20.4489)  2020-06-25T04:52:49.050  

[641 rows x 13 columns]

In [109]:
### add states to cities_df_clean, then merge with disadvantaged score and filter by hazard

## Analyzing City Survey Questions

### Section 1 - Governance

In [ ]:
# 1.4 Does your city incorporate sustainability goals and targets (e.g. GHG reductions) into the master planning for the city?
cities_df[(cities_df['Question Number'] == '1.4')][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)


In [ ]:
# 1.4a Please detail which goals and targets are incorporated in your city’s master plan and describe how these goals are addressed in the table below.
# looking at count of goal types
cities_df[(cities_df['Question Number'] == '1.4a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

In [ ]:
cities_df_1_4a = cities_df[cities_df['Question Number'] == '1.4a']
cities_df_1_4a = cities_df_1_4a[cities_df_1_4a['Response Answer'].notnull()]

cities_df_1_4a.pivot_table(index=['Account Number', 'Organization', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()


<span style="color:red">Potentially use term freqency dictionary to pull out common ways cities plan on addressing goals>

### Section 2 - Hazards and Adaptation 

In [ ]:
#2.0 Has a climate change risk or vulnerability assessment been undertaken for your local government area?
# looking at how many cities answered yes, no, in progress etc. 
cities_df[(cities_df['Question Number'] == '2.0')][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)


In [ ]:
# 2.2a Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity. 
# Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.

cities_df_2_2a = cities_df[cities_df['Question Number'] == '2.2a']
cities_df_2_2a = cities_df_2_2a[cities_df_2_2a['Response Answer'].notnull()]

cities_df_2_2a.pivot_table(index=['Account Number', 'Organization', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()

<span style="color:red">Great question to create some variables here, example could be combining low elevation coastal areas with cities that mentioned drought as a high probability occurance</span>

In [ ]:
# 2.2a Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity. 
# Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.

count_frame = cities_df[(cities_df['Question Number'] == '2.2a') & (cities_df['Row Number'] == 1) & (cities_df['Column Number'] == 1)].groupby('Response Answer').count()
count_frame[['Organization']].sort_values('Organization',ascending=False)

In [ ]:
# 2.2a Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity. 
# Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.

cities_df[(cities_df['Question Number'] == '2.2a') & (cities_df['Column Number'] == 9)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

In [ ]:
#2.4 Please identify the factors that most greatly affect your city’s ability to adapt to climate change, and indicate how those factors either enhance or challenge this ability.

cities_df_2_4 = cities_df[cities_df['Question Number'] == '2.4']
cities_df_2_4 = cities_df_2_4[cities_df_2_4['Response Answer'].notnull()]

cities_df_2_4_wide = cities_df_2_4.pivot_table(index=['Account Number', 'Organization', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
cities_df_2_4_wide

In [ ]:
#2.4 Please identify the factors that most greatly affect your city’s ability to adapt to climate change, and indicate how those factors either enhance or challenge this ability.

cities_df_2_4_wide[cities_df_2_4_wide['Enhance/ Challenge'] == 'Challenge'][['Organization','Factors that affect ability to adapt']].groupby('Factors that affect ability to adapt').count().sort_values('Organization',ascending=False)

<span style="color:red">Similar to 2.2a, great question for variables, perhaps combining low income cities in comparison to country mean with ones that marked poverty or inequality as a challenge factor; also look into consumerism habits of people that are not in poverty; what level of income / spending promotes environmentalism vs consumerism</span>

### Section 4 - Social Risks

In [ ]:
# 4.0 Does your city face any social risks as a result of climate change?
cities_df[(cities_df['Question Number'] == '4.0') & (cities_df['Response Answer'] == 'Yes')][['Organization']].count()

In [ ]:
# 4.0a Please complete the table below to indicate which social risks your city faces as a result of climate change and indicate if these risks have been covered by your city’s action plan.
# looking at most common social risks
cities_df[(cities_df['Question Number'] == '4.0a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

<span style="color:red">This could also be combined with a variety of other 'disadvantaged groups' variables</span>

### Opportunities

In [ ]:
# 5.0a Please indicate the opportunities and describe how the city is positioning itself to take advantage of them.
cities_df[(cities_df['Question Number'] == '5.0a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

In [ ]:
# 5.1 Does your city collaborate in voluntary partnership with businesses in your city on sustainability projects?
cities_df[(cities_df['Question Number'] == '5.1') & (cities_df['Response Answer'] == 'Yes')][['Organization']].count()


In [ ]:
# 5.1a Please provide some key examples of how your city collaborates with business in the table below.
cities_df[(cities_df['Question Number'] == '5.1a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)


<span style="color:red">This could be interesting in matching up with corporations based on industry...</span>

In [ ]:
# 5.2 List any emission reduction, adaptation, water related or resilience projects you have planned within your city for which you hope to attract financing, and provide details on the
cities_df_5_2 = cities_df[cities_df['Question Number'] == '5.2']
cities_df_5_2 = cities_df_5_2[cities_df_5_2['Response Answer'].notnull()]

cities_df_5_2 = cities_df_5_2.pivot_table(index=['Account Number', 'Organization', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
cities_df_5_2.head()

In [ ]:
# 5.2 List any emission reduction, adaptation, water related or resilience projects you have planned within your city for which you hope to attract financing, and provide details on the

cities_df[(cities_df['Question Number'] == '5.2') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

<span style="color:red">Something randomly thought of, could be interesting to match up projects from this question with hazards the city is facing as a way to generate project ideas?? Also projects partially funded match with corporations in that industry</span>

### Local Government Emissions

In [ ]:
#6.0 Do you have an emissions inventory for your local government operations to report?
cities_df[(cities_df['Question Number'] == '6.0') & (cities_df['Column Number'] == 0)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)
